In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import compute_class_weight
import matplotlib.pyplot as plt

In [15]:
# widths = []
# heights = []

# for img_name in os.listdir("Cropped/UTKFace"):
#     if img_name.endswith(".jpg"):
#         img_path = os.path.join("Cropped/UTKFace", img_name)
#         im = Image.open(img_path)
#         widths.append(im.size[0])
#         heights.append(im.size[1])

# AVG_HEIGHT = round(sum(heights)/len(heights))
# AVG_WIDTH = round(sum(widths)/len(widths))

In [16]:
# print(AVG_HEIGHT, AVG_WIDTH)

200 200


In [5]:
data = []
for filename in os.listdir("Cropped/UTKFace"):
    if filename.endswith(".jpg"):
        row_data = [os.path.join("Cropped/UTKFace", filename)]
        filename = filename.split("_")
        if len(filename) > 3:
            # TODO REPLACE INT WITH NAME
            row_data.append(filename[2]) # 3rd integer represents race
            data.append(row_data)

In [26]:
df_image = pd.DataFrame(data, columns = ["filename", "Race"])

df_image = df_image[df_image["Race"] != "4"]

train, test = train_test_split(df_image, stratify = df_image["Race"], test_size=0.2)

# train = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255).flow_from_dataframe(
#     dataframe = train,
#     x_col = "filename",
#     y_col = "Race",
#     class_mode = "categorical",
#     target_size = (200, 200),
#     fill_mode = "nearest",
#     shuffle=True
# )

train = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255,
                                                        fill_mode="nearest",
                                                        horizontal_flip=True,
                                                        brightness_range = (0.4, 0.6)).flow_from_dataframe(
    dataframe = train,
    x_col = "filename",
    y_col = "Race",
    class_mode = "categorical",
    target_size = (200, 200),
    shuffle=True
)

test = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255).flow_from_dataframe(
    dataframe = test,
    x_col = "filename",
    y_col = "Race",
    class_mode = "categorical",
    fill_mode = "nearest",
    target_size=(200, 200)
)
    
# batch = next(train)
# a = np.array(batch[0])

# img = Image.fromarray(a[0].astype('uint8'), "RGB")
# img.save(f"test.png")
# img.show()

Found 17610 validated image filenames belonging to 4 classes.
Found 4403 validated image filenames belonging to 4 classes.


In [7]:
df_image["Race"].value_counts(normalize=True)

Race
0    0.457820
1    0.205606
3    0.180575
2    0.155999
Name: proportion, dtype: float64

In [43]:
compute_class_weight(
    "balanced", 
    classes = np.array(["0", "1", "2", "3"]),
    y = df_image["Race"])

# compute_class_weight(
#     "balanced", 
#     classes = np.array(["0", "1"]),
#     y = df_image["Race"])
# tf.config.list_physical_devices("GPU")

array([0.54606569, 1.21591913, 1.60257717, 1.38446541])

In [27]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(200, 200, 3)))
model.add(keras.layers.Dense(512, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(300, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(128, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(4, activation = "softmax"))
model.compile(loss = "CategoricalCrossentropy", metrics = ["accuracy"], optimizer=keras.optimizers.Adam())
model.fit(train,
          class_weight={0:0.5476668 , 1:1.2282251, 2:1.56114055, 3:1.39017879},
          validation_data = test,
          batch_size = 16,
          epochs = 20)

Epoch 1/20
 78/551 [===>..........................] - ETA: 18s - loss: 11.4449 - accuracy: 0.2563

KeyboardInterrupt: 

In [28]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (5, 5), padding = "same", input_shape=(200, 200, 3), activation = "relu"))
model.add(keras.layers.Conv2D(32, (5, 5), padding = "same", activation = "relu"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(keras.layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(keras.layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation = "relu"))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(4, activation = "softmax"))

model.compile(loss = "CategoricalCrossentropy", metrics = ["accuracy"], optimizer=keras.optimizers.Adam())
model.fit(train,
          validation_data = test,
          class_weight={0:0.54606569, 1:1.21591913 , 2:1.60257717, 3:1.38446541},
          batch_size = 64,
          epochs = 30)

Epoch 1/30
551/551 [==============================] - 62s 111ms/step - loss: 1.2992 - accuracy: 0.4504 - val_loss: 0.9588 - val_accuracy: 0.6393
Epoch 2/30
551/551 [==============================] - 60s 108ms/step - loss: 0.8963 - accuracy: 0.5831 - val_loss: 1.6436 - val_accuracy: 0.3205
Epoch 3/30
551/551 [==============================] - 59s 108ms/step - loss: 0.7950 - accuracy: 0.6302 - val_loss: 1.6262 - val_accuracy: 0.4163
Epoch 4/30
551/551 [==============================] - 59s 108ms/step - loss: 0.7281 - accuracy: 0.6672 - val_loss: 0.6277 - val_accuracy: 0.7767
Epoch 5/30
551/551 [==============================] - 61s 110ms/step - loss: 0.6545 - accuracy: 0.6999 - val_loss: 0.7364 - val_accuracy: 0.7163
Epoch 6/30
551/551 [==============================] - 60s 109ms/step - loss: 0.6143 - accuracy: 0.7149 - val_loss: 0.7868 - val_accuracy: 0.7118
Epoch 7/30
551/551 [==============================] - 61s 112ms/step - loss: 0.5815 - accuracy: 0.7248 - val_loss: 1.0030 - val_ac

In [37]:
model.save("cnn_model")

INFO:tensorflow:Assets written to: cnn_model\assets


INFO:tensorflow:Assets written to: cnn_model\assets


In [42]:
img = Image.open(
    r"C:\Users\Allen\Downloads\IMG_0791.jpg"
)

img = img.resize((200, 200))
img_array = keras.utils.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)
predicted = model.predict(img_array)
# print(np.argmax(predicted, axis = 1)[0])
print(predicted)

1/1 [==============================] - 0s 16ms/step
[[9.9998045e-01 1.5736923e-10 1.3597108e-10 1.9519108e-05]]


In [5]:
import tensorflow as tf
import keras as keras
model = tf.keras.models.load_model(r"C:\Users\Allen\Desktop\Coding\CSC 466\FacialRecognitionRacialBias\cnn_model")

In [6]:
model.save("cnn_model.keras")